In [1]:
from urllib.request import urlopen, urljoin
from urllib import error
from urllib.parse import urlparse

from urllib.robotparser import RobotFileParser

import requests
from bs4 import BeautifulSoup
import re
import os

import pandas as pd
from pandas import DataFrame
import numpy as np
import json
import time
from collections import Counter

In [2]:
import os  

def file_name(file_dir):   
    out = []
    for root, dirs, files in os.walk(file_dir):  
        for file in files:
            if '.json' in file:
                out.append(file)
    return out

In [3]:
output_old = []
allrecipes_list = ['allrecipes.json']
# allrecipes_list =  file_name('Recipes')
# print(len(allrecipes_list))
# print(allrecipes_list)

In [4]:
length = 0
for i in range(len(allrecipes_list)):
#     with open('Recipes/'+allrecipes_list[i].split('|')[0]+
#               '/'+allrecipes_list[i].split('.json')[0]+'/'+allrecipes_list[i]
#               , mode = 'r', encoding = 'utf-8') as f:
    with open(allrecipes_list[i], mode='r', encoding= 'utf-8') as f:
        output_old.append(json.load(f))
        print(len(output_old[i]))
        length = length + len(output_old[i])
        print(length)

26585
26585


In [5]:
'''this function is to merge separate json files into one json file and another csv file'''
def csvjson(allrecipes):

    output  = []
    table = pd.DataFrame(index = [
                            'model','recipe_id','recipe_name','servings','group_name','calories',
                            'protein','carbohydrates','fat','ingredients','recipe_source'
                        ])
            
    '''for allrecipes.com'''
    count = 0
    previous_name = []
    for i in range(len(allrecipes)):
        replace = ''
        for recipeid in range(len(allrecipes[i])):
            replace = allrecipes[i][recipeid]['group_name']
            print('origin: '+allrecipes[i][recipeid]['group_name'])
            if allrecipes[i][recipeid]['group_name'] == 'breakfast':
                replace = 'Breakfast and Brunch Recipes'
            if allrecipes[i][recipeid]['group_name'] == 'lunch':
                replace = 'Lunch and Dinner Recipes'
            if allrecipes[i][recipeid]['group_name'] == 'dinner':
                replace = 'Lunch and Dinner Recipes'
            if allrecipes[i][recipeid]['group_name'] == 'World Cuisine|not known':
                replace = 'World Cuisine'
            print('replace: '+replace)
            
            if allrecipes[i][recipeid]['recipe_name'] in previous_name:
                index = previous_name.index(allrecipes[i][recipeid]['recipe_name'])
                output[index]['group_name'] = output[index]['group_name'] + '|' + allrecipes[i][recipeid]['group_name']
#                 table[replace +'-id'+str(count)]['group_name'] = table[replace +'-id'+str(index)]['group_name'] + '|' + allrecipes[i][recipeid]['group_name']
                continue
            previous_name.append(allrecipes[i][recipeid]['recipe_name'])
            
            count = count + 1
            data = {'model': 'recipes.recipe',
                    'recipe_id': count,
                    'recipe_name': allrecipes[i][recipeid]['recipe_name'],
                    'servings': int(allrecipes[i][recipeid]['servings']),
#                     'group_name': allrecipes[i][recipeid]['group_name'],
                    'group_name': replace,
                    'calories': float(allrecipes[i][recipeid]['calories']),
                    'protein': float(allrecipes[i][recipeid]['protein']),
                    'carbohydrates': float(allrecipes[i][recipeid]['carbohydrates']),
                    'fat': float(allrecipes[i][recipeid]['fat']),
                    'ingredients': allrecipes[i][recipeid]['ingredients'],
                    'recipe_source': allrecipes[i][recipeid]['recipe_source'],
                   }
            
            temp = []
            for s in set(data['group_name'].split('|')):
                temp.append(s.strip())
            data['group_name'] = '|'.join(temp)
                
            output.append(data)     

#             table[allrecipes[i][recipeid]['group_name']+'-id'+str(count)] = [
            table[replace +'-id'+str(count)] = [
                'recipes.recipe',count,allrecipes[i][recipeid]['recipe_name'],int(allrecipes[i][recipeid]['servings']),
#                 allrecipes[i][recipeid]['group_name'],
                replace,
                float(allrecipes[i][recipeid]['calories']),
                float(allrecipes[i][recipeid]['protein']),
                float(allrecipes[i][recipeid]['carbohydrates']),
                float(allrecipes[i][recipeid]['fat']),allrecipes[i][recipeid]['ingredients'],
                allrecipes[i][recipeid]['recipe_source']         
            ]
    
    table = table.stack()
    table = table.unstack(0)
    table.to_csv('allrecipes-no-duplicated.csv')
        
    print('Save as JSON file')
    with open('allrecipes-no-duplicated.json', mode='w', encoding='utf-8') as f:
        json.dump(output, f)

In [6]:
csvjson(output_old) 
with open('allrecipes-no-duplicated.json', mode='r', encoding='utf-8') as f:
    output = json.load(f)
print(len(output))

origin: Chicken Thigh Recipes|Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes|Lunch and Dinner Recipes
replace: Chicken Thigh Recipes|Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes|Lunch and Dinner Recipes
origin: Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes
replace: Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes
origin: Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes|Lunch and Dinner Recipes
replace: Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes|Lunch and Dinner Recipes
origin: Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes
replace: Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes
origin: Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes|Lunch and Dinner Recipes
replace: Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes|Lunch and Dinner Recipes
origin: Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes
replace: Soup, Stew & Chili Recipes|Chicken and Dumpling Recipes
origin: Low Cholesterol Recipes|

replace: ItalianFood|World Cuisine|Soup, Stew & Chili Recipes|Cabbage Soup Recipes
origin: Soup, Stew & Chili Recipes|Cabbage Soup Recipes
replace: Soup, Stew & Chili Recipes|Cabbage Soup Recipes
origin: Soup, Stew & Chili Recipes|Cabbage Soup Recipes
replace: Soup, Stew & Chili Recipes|Cabbage Soup Recipes
origin: Soup, Stew & Chili Recipes|Cabbage Soup Recipes
replace: Soup, Stew & Chili Recipes|Cabbage Soup Recipes
origin: Soup, Stew & Chili Recipes|Cabbage Soup Recipes|Lunch and Dinner Recipes
replace: Soup, Stew & Chili Recipes|Cabbage Soup Recipes|Lunch and Dinner Recipes
origin: Soup, Stew & Chili Recipes|Cabbage Soup Recipes
replace: Soup, Stew & Chili Recipes|Cabbage Soup Recipes
origin: Soup, Stew & Chili Recipes|Cabbage Soup Recipes|Tomato Soup Recipes
replace: Soup, Stew & Chili Recipes|Cabbage Soup Recipes|Tomato Soup Recipes
origin: Soup, Stew & Chili Recipes|Cabbage Soup Recipes
replace: Soup, Stew & Chili Recipes|Cabbage Soup Recipes
origin: Soup, Stew & Chili Recipes|C

origin: Bread Recipes|Muffin Recipes
replace: Bread Recipes|Muffin Recipes
origin: Bread Recipes|Muffin Recipes|Leftover Recipes|Cooking Style
replace: Bread Recipes|Muffin Recipes|Leftover Recipes|Cooking Style
origin: Bread Recipes|Muffin Recipes
replace: Bread Recipes|Muffin Recipes
origin: Bread Recipes|Muffin Recipes
replace: Bread Recipes|Muffin Recipes
origin: Bread Recipes|Muffin Recipes|Banana Bread Recipes
replace: Bread Recipes|Muffin Recipes|Banana Bread Recipes
origin: Bread Recipes|Muffin Recipes
replace: Bread Recipes|Muffin Recipes
origin: Bread Recipes|Muffin Recipes
replace: Bread Recipes|Muffin Recipes
origin: Bread Recipes|Muffin Recipes
replace: Bread Recipes|Muffin Recipes
origin: Breakfast and Brunch Recipes|Bread Recipes|Muffin Recipes
replace: Breakfast and Brunch Recipes|Bread Recipes|Muffin Recipes
origin: Bread Recipes|Muffin Recipes
replace: Bread Recipes|Muffin Recipes
origin: Bread Recipes|Muffin Recipes
replace: Bread Recipes|Muffin Recipes
origin: Bread

replace: Air Fryer Recipes|Sandwich Recipes|Cooking Style|Lunch and Dinner Recipes
origin: Breakfast and Brunch Recipes|Sandwich Recipes|Lunch and Dinner Recipes
replace: Breakfast and Brunch Recipes|Sandwich Recipes|Lunch and Dinner Recipes
origin: Sandwich Recipes|Leftover Recipes|Cooking Style|Lunch and Dinner Recipes
replace: Sandwich Recipes|Leftover Recipes|Cooking Style|Lunch and Dinner Recipes
origin: Sandwich Recipes|Lunch and Dinner Recipes
replace: Sandwich Recipes|Lunch and Dinner Recipes
origin: Sandwich Recipes|Lunch and Dinner Recipes
replace: Sandwich Recipes|Lunch and Dinner Recipes
origin: Sandwich Recipes|Sandwich Wraps and Roll-Up Recipes|Lunch and Dinner Recipes
replace: Sandwich Recipes|Sandwich Wraps and Roll-Up Recipes|Lunch and Dinner Recipes
origin: Sandwich Recipes|Lunch and Dinner Recipes
replace: Sandwich Recipes|Lunch and Dinner Recipes
origin: Sandwich Recipes|Lunch and Dinner Recipes
replace: Sandwich Recipes|Lunch and Dinner Recipes
origin: Sandwich Rec

replace: Breakfast and Brunch Recipes|Diet & Healthy Recipes|Keto Recipes|Cooking Style|Air Fryer Recipes
origin: Pork Chop Recipes|Diet & Healthy Recipes|Keto Recipes|Main Dish Recipes|Cooking Style|Air Fryer Recipes|Lunch and Dinner Recipes
replace: Pork Chop Recipes|Diet & Healthy Recipes|Keto Recipes|Main Dish Recipes|Cooking Style|Air Fryer Recipes|Lunch and Dinner Recipes
origin: Diet & Healthy Recipes|Keto Recipes|Lunch and Dinner Recipes
replace: Diet & Healthy Recipes|Keto Recipes|Lunch and Dinner Recipes
origin: Diet & Healthy Recipes|Keto Recipes
replace: Diet & Healthy Recipes|Keto Recipes
origin: Diet & Healthy Recipes|Keto Recipes|Dessert Recipes|Sugar-Free Dessert Recipes
replace: Diet & Healthy Recipes|Keto Recipes|Dessert Recipes|Sugar-Free Dessert Recipes
origin: Breakfast and Brunch Recipes|Diet & Healthy Recipes|Keto Recipes
replace: Breakfast and Brunch Recipes|Diet & Healthy Recipes|Keto Recipes
origin: Diet & Healthy Recipes|Keto Recipes|Lunch and Dinner Recipes


...

origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Rec

replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Rec

origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Rec

replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Recipes
replace: Lunch and Dinner Recipes
origin: Lunch and Dinner Rec

Save as JSON file
26585


In [7]:
len(output)

26585